# Project Insurance Domain | Car accident risk assessments

Here is the paper containg the datasets : https://dutangc.perso.math.cnrs.fr/RRepository/pub/web/CASdatasets-manual.pdf#page=56&zoom=100,133,256

## Datasets description 
Datasets from the same database of an unknown private motor portfolio observed between January 2003 and December 2004

respectively :

- freq -> claim frequency databases
- sev -> claim severity databases
- prem -> premium databases

Here is the description for each dataset :
**prem**

| Variable       | Description |
| --------       | ----------- |
| IDpol          | Le numéro d'identifiant de la police |
| DrivAge        | L'âge du conducteur, en années |
| DriveGener     | Le sexe du conducteur ("facteur") |
| MaritalStatus  | Le statut marital ("facteur") |
| BonusMalus     | Le bonus-malus |
| LicenceNb      | Le numéro de permis de conduire |
| Payfreq        | La fréquence des paiements ("facteur") |
| JobCode        | Un code correspondant à la profession ("facteur") |
| VehAge         | L'âge du véhicule, en années |
| VehClass       | Le type de véhicule ("facteur") |
| VehPower       | La puissance du véhicule ("facteur") du moins puissant "P2" au plus puissant "P15" |
| VehGas         | Le type de moteur : diesel ou essence ("facteur") |
| VehUsage       | L'usage du véhicule ("facteur") |
| Garage         | Le type de garage ("facteur") |
| Area           | The area code (as "factor"): unknown category |
| Region         | The policy regions in France (based on a standard French classification) |
| Channel        | The channel distribution code (as "factor"): unknown category |
| Marketing      | The marketing code (as "factor"): unknown category |
| PremWindscreen | The premium for windscreen guarantee (annual basis) |
| PremDamAll     | The premium for damage all-accident guarantee (annual basis) |
| PremFire       | The premium for fire guarantee (annual basis) |
| PremAcc1       | The premium for type-1 accident guarantee (annual basis) |
| PremAcc2       | The premium for type-2 accident guarantee (annual basis) |
| PremLegal      | The premium for legal protection guarantee (annual basis) |
| PremTPLV       | The premium for voluntary third-part liability guarantee (annual basis) |
| PremTPLM       | The premium for mandatory third-part liability guarantee (annual basis) |
| PremServ       | The premium for service guarantee (annual basis) |
| PremTheft      | The premium for theft guarantee (annual basis) |
| PremTot        | The total premium (annual basis) |
| Year           | Numeric for the year |


**freq**

| Variable | Description |
| -------- | ----------- |
| IDpol | The policy ID (used to link with the claims dataset) |
| Year | The underwriting year |
| Damage | The claim number for the Damage guarantee |
| Fire | The claim number for the Fire guarantee |
| Other | The claim number for the Other guarantee |
| Theft | The claim number for the Theft guarantee |
| TPL | The claim number for the TPL guarantee |
| Windscreen | The claim number for the Windscreen guarantee |

**sev**

| Variable | Description |
| -------- | ----------- |
| IDpol | The policy ID (used to link with the claims dataset) |
| OccurDate | The occurence date |
| Payment | The amount of money paid |
| IDclaim | The claim ID |
| Guarantee | The corresponding guarantee of the claim |


## Project
First of all let's do useful imports suchs as numpy and pandas for calculation and data manipulation or matplotlib and seaborn for nice graphs and DataViz

In [3]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
SEED = 42

In [4]:
df = pd.read_csv('./data/all_data.csv', index_col=[0])

/tmp/ipykernel_22598/363754127.py:1: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/all_data.csv', index_col=[0])


Quick data validation (verify NaN and inconsistencies)

In [5]:
nan_counts = df.isna().sum()
# Filter columns with NaN values
columns_with_nan = nan_counts[nan_counts > 0]
columns_with_nan

MaritalStatus    23246
JobCode          23246
OccurDate        25307
Payment          25307
IDclaim          25307
Guarantee        25307
dtype: int64

In [6]:
for i in columns_with_nan.keys():
    print('Treating column : ', i)
    print('Number of missing values : ', df[i].isna().sum())
    print(df[i].value_counts())
    print('\n')

Treating column :  MaritalStatus
Number of missing values :  23246
MaritalStatus
Cohabiting    7105
Married       2365
Single        1361
Widowed        340
Divorced       136
Name: count, dtype: int64


Treating column :  JobCode
Number of missing values :  23246
JobCode
Private employee    6091
Public employee     3338
Retiree              690
Other                565
Craftsman            438
Farmer               167
Retailer              18
Name: count, dtype: int64


Treating column :  OccurDate
Number of missing values :  25307
OccurDate
2003-01-20    28
2004-07-16    27
2003-05-18    26
2004-11-30    26
2004-09-25    25
              ..
2003-12-26     3
2003-08-29     2
2004-05-05     2
2003-07-25     2
2004-06-09     2
Name: count, Length: 722, dtype: int64


Treating column :  Payment
Number of missing values :  25307
Payment
897.0     448
0.0       328
915.0     215
1121.0     92
199.0      34
         ... 
1625.0      1
806.0       1
4373.0      1
4288.0      1
2571.0      1


In [7]:
for i in ['MaritalStatus', 'JobCode']:
    df[i].fillna('Unknown', inplace = True)

In [8]:
nan_counts = df.isna().sum()
# Filter columns with NaN values
columns_with_nan = nan_counts[nan_counts > 0]
columns_with_nan

OccurDate    25307
Payment      25307
IDclaim      25307
Guarantee    25307
dtype: int64

In [9]:
df['GotClaim'] = df['Payment'].notna()
df.drop(columns = ['OccurDate', 'Payment', 'IDclaim', 'Guarantee', 'Damage', 'Fire', 'Other', 'Theft', 'TPL', 'Windscreen'], inplace=True)

These missing values are 'normal' it means that there is no Claim for the corresponding PolicyID.

Let's create our target feature which is whether a person got claim or not

In [10]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
test_df.drop(columns=['GotClaim'], inplace=True)


In [11]:
train_df.to_csv('./data/train.csv')
test_df.to_csv('./data/test.csv')

In [12]:
train = pd.read_csv('./data/train.csv', index_col=[0])
test = pd.read_csv('./data/test.csv', index_col=[0])

In [13]:
train.head()

,IDpol,DrivAge,DrivGender,MaritalStatus,BonusMalus,LicenceNb,PayFreq,JobCode,VehAge,VehClass,...,PremFire,PremAcc1,PremAcc2,PremLegal,PremTPLM,PremTPLV,PremServ,PremTheft,PremTot,GotClaim
22083,90158529.100b,29.0,F,Unknown,64.0,2.0,Half-yearly,Unknown,6.0,Cheaper,...,9.0,37.0,0.0,12.0,182.5,10.0,57.0,88.0,419.5,True
23319,90168155.100a,30.0,F,Unknown,90.0,1.0,Annual,Unknown,9.0,Cheaper,...,0.0,36.0,0.0,15.0,352.0,19.0,55.0,0.0,477.0,False
30662,90147484.101a,42.0,M,Unknown,50.0,2.0,Half-yearly,Unknown,3.0,Medium low,...,13.0,0.0,0.0,17.0,224.9,12.0,57.0,105.0,760.9,False
17190,90131741.100b,29.0,M,Unknown,85.0,2.0,Half-yearly,Unknown,14.0,Cheaper,...,0.0,35.0,0.0,10.0,196.6,0.0,58.0,0.0,299.6,True
11823,90185369.101a,44.0,M,Unknown,50.0,2.0,Annual,Unknown,6.0,Medium low,...,6.0,0.0,0.0,10.0,124.2,6.0,50.0,76.0,455.2,False


In [14]:
train.columns

Index(['IDpol', 'DrivAge', 'DrivGender', 'MaritalStatus', 'BonusMalus',
       'LicenceNb', 'PayFreq', 'JobCode', 'VehAge', 'VehClass', 'VehPower',
       'VehGas', 'VehUsage', 'Garage', 'Area', 'Region', 'Channel',
       'Marketing', 'PremWindscreen', 'PremDamAll', 'PremFire', 'PremAcc1',
       'PremAcc2', 'PremLegal', 'PremTPLM', 'PremTPLV', 'PremServ',
       'PremTheft', 'PremTot', 'GotClaim'],
      dtype='object')

Format data (int to int, categorical variables and so on...)

In [21]:
numeric_features = ['DrivAge', 'VehAge', 'BonusMalus', 'PremWindscreen', 'PremDamAll', 'PremFire', 'PremAcc1', 'PremAcc2', 
                     'PremLegal', 'PremTPLM', 'PremTPLV', 'PremServ', 'PremTheft']

categorical_features = ['DrivGender', 'MaritalStatus', 'LicenceNb', 'PayFreq', 'JobCode', 'VehClass', 'VehPower', 
                       'VehGas', 'VehUsage', 'Garage', 'Area', 'Region', 'Channel', 'Marketing']

In [22]:
train.dtypes

IDpol              object
DrivAge           float64
DrivGender         object
MaritalStatus      object
BonusMalus        float64
LicenceNb         float64
PayFreq            object
JobCode            object
VehAge            float64
VehClass           object
VehPower           object
VehGas             object
VehUsage           object
Garage             object
Area               object
Region             object
Channel            object
Marketing          object
PremWindscreen    float64
PremDamAll        float64
PremFire          float64
PremAcc1          float64
PremAcc2          float64
PremLegal         float64
PremTPLM          float64
PremTPLV          float64
PremServ          float64
PremTheft         float64
PremTot           float64
GotClaim             bool
dtype: object

In [23]:
y = train['GotClaim']
X = train.drop(columns=['GotClaim'])

# Prepare the Machine Learning Pipelines

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.datasets import fetch_openml
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline

In [25]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [26]:
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.740


/home/mnedellec/.virtualenvs/mael_spotify/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
